In [1]:
import pandas as pd 
import vaex
import seaborn as sns
import pyodbc

# Data Import

In [2]:
transactions_df = vaex.open('transactions_train.csv')
customers_df = vaex.open('customers.csv')
articles_df = vaex.open('articles.csv')

In [3]:
transactions_df.head(5)

#,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...,663713001,0.0508305,2
1,2018-09-20,'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...,541518023,0.0304915,2
2,2018-09-20,'00007d2de826758b65a93dd24ce629ed66842531df66993...,505221004,0.0152373,2
3,2018-09-20,'00007d2de826758b65a93dd24ce629ed66842531df66993...,685687003,0.0169322,2
4,2018-09-20,'00007d2de826758b65a93dd24ce629ed66842531df66993...,685687004,0.0169322,2


In [4]:
customers_df.head(5)

#,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,'00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d...,nan,nan,ACTIVE,NONE,49,'52043ee2162cf5aa7ee79974281641c6f11a68d276429a9...
1,'0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee...,nan,nan,ACTIVE,NONE,25,'2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93f...
2,'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...,nan,nan,ACTIVE,NONE,24,'64f17e6a330a85798e4998f62d0930d14db8db1c054af6c...
3,'00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c...,nan,nan,ACTIVE,NONE,54,'5d36574f52495e81f019b680c843c443bd343d5ca5b1c22...
4,'00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe...,1,1,ACTIVE,Regularly,52,'25fa5ddee9aac01b35208d01736e57942317d756b32ddd4...


In [5]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=.\sqlexpress;"
            "Database=h_m;"
            "Trusted_Connection=yes;")

cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()


query = pd.read_sql(
"""

;with p as (
SELECT distinct product_code,prod_name
 FROM [h_m].[dbo].[articles]),

d as (
select p.product_code,count(*) cnt
from p 
group by p.product_code
having count(*) >1),

s as (
select product_code
from p
group by product_code
having count(*) = 1
)

select x.product_code,x.prod_name
from(
select p.*, ROW_NUMBER() over (partition by p.product_code order by p.product_code desc) rn
from d
join p
on d.product_code = p.product_code
)x
where x.rn = 1
union 
select p.*
from s
join p
on s.product_code = p.product_code""", cnxn)

product = vaex.from_pandas(query)

In [6]:
articles_df.head(3)

#,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,1,Dusty Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


# Data Cleaning

In [7]:
transactions_df.rename('t_dat', 'date')

'date'

In [8]:
articles_df = articles_df.drop(columns=['product_type_no', 'graphical_appearance_no','colour_group_code','perceived_colour_value_id','perceived_colour_master_id','department_no','section_no','garment_group_no','index_group_no','index_code','prod_name'])

In [9]:
articles_df.rename('product_code', 'prod_id')

'prod_id'

In [10]:
product['product_code']=product.product_code.astype('int64')

In [11]:
articles_df = product.join(articles_df, how='inner', left_on ='product_code', right_on='prod_id',allow_duplication=True)

In [12]:
articles_df = articles_df.drop(columns=['prod_id','product_code'])

In [14]:
df = transactions_df.join(articles_df, how='inner', left_on ='article_id',right_on='article_id')

In [15]:
df = df.join(customers_df, how='inner', left_on ='customer_id', right_on='customer_id')

In [16]:
df['date'] = df['date'].astype('datetime64')
df = df.drop(columns=(['postal_code']))

# Exploratory Data Analysis

In [17]:
print('transactions',transactions_df.shape)
print('customers',customers_df.shape)
print('articles',articles_df.shape)
print('main',df.shape)

transactions (31788324, 5)
customers (1371980, 7)
articles (105542, 14)
main (31788324, 23)


In [43]:
count_na = []  # to count the missing value per column
for col in df.column_names:
    count_na.append(df[col].isna().sum().item())

s = pd.Series(data=count_na, index=df.column_names).sort_values(ascending=True)

In [44]:
s

date                                   0
garment_group_name                     0
section_name                           0
index_group_name                       0
index_name                             0
department_name                        0
perceived_colour_master_name           0
colour_group_name                      0
graphical_appearance_name              0
perceived_colour_value_name            0
product_type_name                      0
prod_name                              0
__product_code                         0
sales_channel_id                       0
price                                  0
article_id                             0
customer_id                            0
__date                                 0
product_group_name                     0
club_member_status                 62165
detail_desc                       115027
age                               140258
fashion_news_frequency            141711
FN                              18209837
Active          

In [30]:
df.groupby('prod_name').agg('count').sort('count',ascending=False).head(10)

#,prod_name,count
0,Jade Denim HW trs,183921
1,Luna skinny 5 pkt,158557
2,Tilly,137799
3,Timeless Midrise Brief,122143
4,Shake it in Balconette,83924
5,Cat Tee.,81304
6,Despacito,78249
7,Gyda blouse,75867
8,Simple as That Triangle Top,74827
9,7p Basic Shaftless,73456


In [31]:
df.groupby('product_group_name').agg('count').sort('count',ascending=False).head(10)

#,product_group_name,count
0,Garment Upper body,12552755
1,Garment Lower body,7046054
2,Garment Full body,3552470
3,Swimwear,2579222
4,Underwear,2565858
5,Accessories,1599593
6,Shoes,745521
7,Socks & Tights,685712
8,Nightwear,348180
9,Unknown,97040


In [32]:
df.groupby('colour_group_name').agg('count').sort('count',ascending=False).head(10)

#,colour_group_name,count
0,Black,11036956
1,White,3368276
2,Dark Blue,2180620
3,Light Beige,1263385
4,Blue,1088455
5,Beige,921184
6,Light Blue,913496
7,Light Pink,858234
8,Off White,841673
9,Grey,840065


In [33]:
df.groupby('department_name').agg('count').sort('count',ascending=False).head(10)

#,department_name,count
0,Swimwear,2495122
1,Trouser,1742298
2,Blouse,1651060
3,Knitwear,1598097
4,Jersey,1526503
5,Jersey Basic,1469357
6,Expressive Lingerie,1196062
7,Jersey fancy,1156697
8,Basic 1,1140599
9,Dress,1114750


In [34]:
df.groupby('section_name').agg('count').sort('count',ascending=False).head(10)

#,section_name,count
0,Womens Everyday Collection,5860832
1,Divided Collection,3768107
2,"Womens Swimwear, beachwear",2455138
3,Womens Lingerie,2200139
4,Womens Tailoring,2133999
5,Womens Everyday Basics,1541864
6,Divided Basics,1290216
7,Womens Casual,1228424
8,Ladies H&M Sport,1177490
9,"Womens Nightwear, Socks & Tigh",1110181


In [35]:
df.groupby('garment_group_name').agg('count').sort('count',ascending=False).head(10)

#,garment_group_name,count
0,Jersey Fancy,5171611
1,Jersey Basic,3268451
2,"Under-, Nightwear",2966788
3,Trousers,2953711
4,Swimwear,2538085
5,Blouses,2510441
6,Knitwear,2392807
7,Dresses Ladies,2147529
8,Accessories,1645464
9,Trousers Denim,1279886


In [36]:
df.groupby('index_group_name').agg('count').sort('count',ascending=False).head(10)

#,index_group_name,count
0,Ladieswear,20415260
1,Divided,7138254
2,Menswear,1771053
3,Sport,1246408
4,Baby/Children,1217349


In [37]:
df.groupby('index_name').agg('count').sort('count',ascending=False).head(10)

#,index_name,count
0,Ladieswear,13058283
1,Divided,7138254
2,Lingeries/Tights,5582246
3,Ladies Accessories,1774731
4,Menswear,1771053
5,Sport,1246408
6,Children Sizes 134-170,503507
7,Children Sizes 92-140,350514
8,Baby Sizes 50-98,254137
9,"Children Accessories, Swimwear",109191


seasonal analysis, monthly, yearly,gender, age group 